In [ ]:
import torch
import torch.nn as nn  
import torch.optim as optim 
import torchvision.transforms as transforms 
import torch.nn.functional as F
import torchvision
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader,
) 


In [ ]:
from torch.nn.modules.batchnorm import BatchNorm1d

class FCN(nn.Module):
    def __init__(self,in_channels = 1,num_classes=7):
        super(FCN,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(32, momentum=0.01),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(64, momentum=0.01),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.BatchNorm2d(64, momentum=0.01),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),
            nn.MaxPool2d(kernel_size=2),
        )
        self.fc1 = nn.Sequential(
            nn.Linear(20736,256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.25),
            nn.BatchNorm1d(256)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256,128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.25),
            nn.BatchNorm1d(128)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.25),
            nn.BatchNorm1d(64)
        )
        self.fc4 = nn.Linear(64,7)

    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = F.softmax(self.fc4(x),dim=1)
        return x

In [ ]:
model = FCN()
if torch.cuda.is_available():
    model.cuda()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [ ]:
if torch.cuda.is_available():
    model.cuda()

In [ ]:
import shutil

In [ ]:
# Data Loading
path = './Augmented_inverted.zip'
shutil.unpack_archive(path,'./dataset')

path = './testing_inverted.zip'
shutil.unpack_archive(path,'./testing_dataset')

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from PIL import Image
import numpy as np

In [ ]:
data_transform = transforms.Compose([
                                     transforms.Grayscale(num_output_channels=1),
                                     transforms.Lambda(lambda x:torch.from_numpy(np.expand_dims(np.array(x),axis=0)).float())
                                ])

In [ ]:
root = './dataset'
train_dataset = ImageFolder(root, transform=data_transform)
test_dataset = ImageFolder("./testing_dataset", transform=data_transform)

# Split test and train dataset 

train_size = int(0.9 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_data, test_data = random_split(train_dataset, [train_size, test_size])


# Set batch size of train data loader
batch_size_train = 64

# Set batch size of test data loader
batch_size_test = 64

# load the split train and test data into batches via DataLoader()
train_loader = DataLoader(train_data, batch_size=batch_size_train, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size_test, shuffle=True)

In [ ]:
# Train loop

in_channels = 1
num_classes = 8
learning_rate = 0.01
batch_size = 64
num_epochs = 25

data_transform = transforms.Compose([
                                transforms.Grayscale(num_output_channels=1),
                                transforms.Lambda(lambda x:torch.tensor(np.expand_dims(np.array(x),axis=0))),
                                transforms.RandomRotation(degrees=90),
                                ])

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

model.train()
for epoch in range(num_epochs):
    correct = 0
    for batch_idx, (data,targets) in enumerate(train_loader):

        data = data.to(device=device)
        targets = targets.to(device=device)
        score = model(data)
        loss = criterion(score,targets)
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step()
        
    _, predicted = torch.max(score.data, 1)
    correct += (predicted == targets).float().sum()
    accuracy = 100 * correct / len(targets)
    print("Epoch = {}, Accuracy = {}".format(epoch,accuracy))

data_transform = transforms.Compose([
                                transforms.Grayscale(num_output_channels=1),
                                transforms.Lambda(lambda x:torch.tensor(np.expand_dims(np.array(x),axis=0))),
                                transforms.RandomRotation(degrees=90),
                                ])


in_channels = 1
num_classes = 8
learning_rate = 0.005
batch_size = 64
num_epochs = 25

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

model.train()
for epoch in range(num_epochs):
    correct = 0
    for batch_idx, (data,targets) in enumerate(train_loader):

        data = data.to(device=device)
        targets = targets.to(device=device)
        score = model(data)
        loss = criterion(score,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    _, predicted = torch.max(score.data, 1)
    correct += (predicted == targets).float().sum()
    accuracy = 100 * correct / len(targets)
    print("Epoch = {}, Accuracy = {}".format(epoch,accuracy))

data_transform = transforms.Compose([
                                    transforms.Grayscale(num_output_channels=1),
                                    transforms.Lambda(lambda x:torch.tensor(np.expand_dims(np.array(x),axis=0))),
                                ])

in_channels = 1
num_classes = 8
learning_rate = 0.001
batch_size = 64
num_epochs = 25

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

for epoch in range(num_epochs):
    correct = 0
    for batch_idx, (data,targets) in enumerate(train_loader):

        data = data.to(device=device)
        targets = targets.to(device=device)
        score = model(data)
        loss = criterion(score,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    _, predicted = torch.max(score.data, 1)
    correct += (predicted == targets).float().sum()
    accuracy = 100 * correct / len(targets)
    print("Epoch = {}, Accuracy = {}".format(epoch,accuracy))

in_channels = 1
num_classes = 8
learning_rate = 0.0005
batch_size = 64
num_epochs = 25

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

for epoch in range(num_epochs):
    correct = 0
    for batch_idx, (data,targets) in enumerate(train_loader):

        data = data.to(device=device)
        targets = targets.to(device=device)
        score = model(data)
        loss = criterion(score,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    _, predicted = torch.max(score.data, 1)
    correct += (predicted == targets).float().sum()
    accuracy = 100 * correct / len(targets)
    print("Epoch = {}, Accuracy = {}".format(epoch,accuracy))



In [ ]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Save Model

In [ ]:
torch.save(model.state_dict(),"./model_Inverted")